In [1]:
import gurobipy as grb
import numpy as np
import itertools

In [2]:
S, I, J, K, L, M = 15, 3, 5, 6, 5, 4
plan = [0, 1, 2]
def iterer(*args):
    return itertools.product(*[range(x_) for x_ in args])

In [3]:
model = grb.Model("eurobot")

x = model.addVars(range(S), range(I), range(J), range(K), range(L), range(M), name = 'x', vtype=grb.GRB.BINARY)

## Unique action per step

In [4]:
model.addConstrs((x.sum(s, '*', '*', '*', '*', '*') <= 1 for s in iterer(S)));

## Unique cubes and places constraints

In [5]:
model.addConstrs((x.sum('*', i, '*', '*', l, '*') == 1 for i, l in iterer(I, L)));
model.addConstrs((x.sum('*', '*', j, k, '*', '*') <= 1 for j, k in iterer(J, K)));

## Tower height

In [6]:
#model.addConstrs((x.sum('*', i, '*', '*', '*', '*') == 5 for i in iterer(I)));

## Bonus plan requirements

In [7]:
pi = model.addVars(range(I), range(6), name = 'pi', vtype=grb.GRB.BINARY)

plan_locations = [[0, 1, 2], [1, 2, 3], [2, 3, 4], [4, 3, 2], [3, 2, 1], [2, 1, 0]]
for i, t in iterer(I, 6):
    for j in range(3):
        model.addConstr(pi[i, t] <= x.sum('*', i, plan[j], '*', plan_locations[t][j], '*'))

xi = model.addVars(range(I), name='xi', vtype=grb.GRB.BINARY)
model.addConstrs((xi[i] <= pi.sum(i, '*') for i in range(I)))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

## l-Ordering

In [8]:
for s, l1, i in iterer(S,L,I):
    for l2 in range(l1+1,L):
        model.addConstr(grb.quicksum((x.sum(s_,i,'*','*',l1,'*') for s_ in iterer(s) )) >= x.sum(s,i,'*','*',l2,'*'))

## Ban collisions with other cubes

In [9]:
j_rel = np.zeros((J+1,J+1,2))
j_map = np.array([
                 [-1,-1,-1,-1,-1],
                 [-1,-1, 2,-1,-1],
                 [-1, 3, 4, 1,-1],
                 [-1,-1, 0,-1,-1],
                 [-1,-1,-1,-1,-1]
                 ])

j_on_map = np.zeros((J+1,2))


for i1 in range(j_map.shape[0]):
    for j1 in range(j_map.shape[1]):
        if j_map[i1,j1] >= 0:
            j_on_map[j_map[i1,j1]] = np.array([j1,i1])

for i1 in range(j_map.shape[0]):
    for j1 in range(j_map.shape[1]):
        for i2 in range(j_map.shape[0]):
            for j2 in range(j_map.shape[1]):
                if j_map[i1,j1] >= 0 and j_map[i2,j2] >= 0:
                    if i1 != i2 or j1 != j2:
                        j_rel[j_map[i1,j1],j_map[i2,j2]] = np.array([j2-j1,i2-i1])
                        
# we need just to have free space on 3 'kletka' around the manipulator
def m_manipulator(i,m):
    return (m+i-1)%4

only_non_free_places = [np.array([0,-1]),np.array([-1,0]),np.array([0,1]),np.array([1,0])]

def only_non_free_dif(i,m):
    direction =  m_manipulator(i,m)
    return only_non_free_places[direction]

def our_position(j1):
    return j_on_map[j1]

def list_of_possible_breaks(j1,i,m):
    toreturn = []
    dxy = [np.array([0,1]),np.array([1,0]),np.array([-1,0]),np.array([0,-1])]
    for dd in dxy:
        if (dd != only_non_free_dif(i,m)).any():
            yx = j_on_map[j1] + dd
            if j_map[int(yx[1]),int(yx[0])] >= 0:
                toreturn.append(j_map[int(yx[1]),int(yx[0])])
    return toreturn

list_of_possible_breaks(2, 2, 3)

[4]

In [10]:
def debug_possible_breaks(j1, i, m):
    return [0] if j1 == 4 else []

In [11]:
for s, i, j, k, m in iterer(S, I, J, K, M):
    cubes_to_peak = list_of_possible_breaks(j, i, m)
#     print("%d %d %d"%(i,j,m) + ' ' + str(cubes_to_peak))
    left_gen = (x.sum(s_prev, '*', cube, k, '*', '*') for s_prev, cube in itertools.product(range(s), cubes_to_peak))
    print(grb.quicksum(left_gen))
    left_gen = (x.sum(s_prev, '*', cube, k, '*', '*') for s_prev, cube in itertools.product(range(s), cubes_to_peak))
#         expr = grb.LinExpr()
#         for s_prev, cube in itertools.product(range(s), cubes_to_peak):
#             expr += x.sum(s_prev, '*', cube, k, '*', '*')
    if s> 0 and len(cubes_to_peak) > 0:    
        model.addConstr(grb.quicksum(left_gen) >= x.sum(s, i, j, k, '*', m))

<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: 0.0>
<gurobi.Li

<gurobi.LinExpr: x[0,0,4,2,0,0] + x[0,0,4,2,0,1] + x[0,0,4,2,0,2] + x[0,0,4,2,0,3] + x[0,0,4,2,1,0] + x[0,0,4,2,1,1] + x[0,0,4,2,1,2] + x[0,0,4,2,1,3] + x[0,0,4,2,2,0] + x[0,0,4,2,2,1] + x[0,0,4,2,2,2] + x[0,0,4,2,2,3] + x[0,0,4,2,3,0] + x[0,0,4,2,3,1] + x[0,0,4,2,3,2] + x[0,0,4,2,3,3] + x[0,0,4,2,4,0] + x[0,0,4,2,4,1] + x[0,0,4,2,4,2] + x[0,0,4,2,4,3] + x[0,1,4,2,0,0] + x[0,1,4,2,0,1] + x[0,1,4,2,0,2] + x[0,1,4,2,0,3] + x[0,1,4,2,1,0] + x[0,1,4,2,1,1] + x[0,1,4,2,1,2] + x[0,1,4,2,1,3] + x[0,1,4,2,2,0] + x[0,1,4,2,2,1] + x[0,1,4,2,2,2] + x[0,1,4,2,2,3] + x[0,1,4,2,3,0] + x[0,1,4,2,3,1] + x[0,1,4,2,3,2] + x[0,1,4,2,3,3] + x[0,1,4,2,4,0] + x[0,1,4,2,4,1] + x[0,1,4,2,4,2] + x[0,1,4,2,4,3] + x[0,2,4,2,0,0] + x[0,2,4,2,0,1] + x[0,2,4,2,0,2] + x[0,2,4,2,0,3] + x[0,2,4,2,1,0] + x[0,2,4,2,1,1] + x[0,2,4,2,1,2] + x[0,2,4,2,1,3] + x[0,2,4,2,2,0] + x[0,2,4,2,2,1] + x[0,2,4,2,2,2] + x[0,2,4,2,2,3] + x[0,2,4,2,3,0] + x[0,2,4,2,3,1] + x[0,2,4,2,3,2] + x[0,2,4,2,3,3] + x[0,2,4,2,4,0] + x[0,2,4,2,4,1]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## Time

In [12]:
v = 20
t_p = 2 # time of picking
t_r = 1 # time of rotation on pi / 2
t_m = 2 # time of moving on 6 cm

picking_points = np.zeros((5, 4, 3, 3))
cubes_coords = np.array([[0, -1], [1, 0], [0, 1], [-1, 0], [0, 0]])
l1 = np.array([[[1, 0], [0, 1], [-1, 0]],
               [[0, 1], [-1, 0], [0, -1]],
               [[-1, 0], [0, -1], [1, 0]],
               [[0, -1], [1, 0], [0, 1]]])

picking_points[..., :2] = cubes_coords[:, np.newaxis, np.newaxis, :] + l1[np.newaxis, :, :, :]
picking_points[..., 2] = np.array(range(4))[np.newaxis, :, np.newaxis]

points_2 = np.reshape(picking_points, (60, 3))
small_time_2 = t_m * np.abs(points_2[:, np.newaxis, 0] - points_2[np.newaxis, :, 0]) + \
             t_m * np.abs(points_2[:, np.newaxis, 1] - points_2[np.newaxis, :, 1]) + \
             t_r * ((points_2[:, np.newaxis, 2] - points_2[np.newaxis, :, 1]) % 4)
        
small_time = np.where(small_time_2 == 0, 0, small_time_2 + t_p)
heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])
all_picking_points = np.reshape(points_2[np.newaxis, :, :2] * 6 + heap_points[:, np.newaxis, :2], (360, 2))
big_time = np.sum((heap_points[:, np.newaxis] - heap_points[np.newaxis, :]) ** 2, axis=2) ** 0.5 / v + t_p
times = np.sum((all_picking_points[:, np.newaxis] - all_picking_points[np.newaxis, :]) ** 2, axis=2) ** 0.5 / v + t_p
for i in range(6):
    times[i * 60: (i + 1) * 60, i * 60: (i + 1) * 60] = small_time
small_time = small_time.reshape(5, 4, 3, 5, 4, 3)
times = times.reshape(6, 5, 4, 3, 6, 5, 4, 3)
np.all(times >= 0)

True

In [13]:
%%time
f = model.addVars(range(S), range(I), range(J), range(M), name = 'f', vtype=grb.GRB.BINARY)
model.addConstrs((x.sum(s, i, j, '*', '*', m) == f[s, i, j, m] for s, i, j, m in iterer(S, I, J, M)))
# gen = (f[s, i_1, j_1, m_1] * f[s + 1, i_2, j_2, m_2] * \
#        times[k_1, j_1, m_1, i_1, k_2, j_2, m_2, i_2] for s, i_1, j_1, k_1, m_1, i_2, j_2, k_2, m_2, l_1, l_2 \
#        in iterer(S - 1, I, J, K, M, I, J, K, M, L, L))
gen_1 = (f[s, i_1, j_1, m_1] * f[s + 1, i_2, j_2, m_2] * \
       small_time[j_1, m_1, i_1, j_2, m_2, i_2] for s, i_1, j_1, m_1, i_2, j_2, m_2 \
       in iterer(S - 1, I, J, M, I, J, M))
km = model.addVars(range(S), range(K), name = 'km', vtype=grb.GRB.BINARY)
model.addConstrs((x.sum(s, '*', '*', k, '*', '*') == km[s, k] for s, k in iterer(S, K)))
gen_2 = (km[s, k_1] * km[s, k_2] * \
       big_time[k_1, k_2] for s, k_1, k_2 \
       in iterer(S - 1, K, K)) 
# model.addConstr(grb.quicksum(gen) <= T, 'time')
model.setObjective(grb.quicksum(gen_1) + grb.quicksum(gen_2))

Wall time: 1.1 s


## Picking score

In [ ]:
# obj = xi.sum('*')
# model.setObjective(obj, grb.GRB.MAXIMIZE)
model.addConstr(xi.sum('*')==3)

<gurobi.Constr *Awaiting Model Update*>

## Optimization 

In [ ]:
%%time
model.Params.TimeLimits = 300
model.update()
# model.relax()
model.optimize()

Optimize a model with 5590 rows, 28011 columns and 3329268 nonzeros
Model has 49392 quadratic objective terms
Variable types: 0 continuous, 28011 integer (28011 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 97 rows and 3609 columns (presolve time = 5s) ...
Presolve added 122 rows and 0 columns
Presolve removed 0 rows and 3390 columns
Presolve time: 5.41s
Presolved: 55020 rows, 73929 columns, 2487327 nonzeros
Variable types: 0 continuous, 73929 integer (73710 binary)
Found heuristic solution: objective 74.0000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.800000e+01   0.000000e+00      9s
    2328    2.8000017e+01   3.143577e+02   0.000000e+00     10s
    3430    2.8000000e+01   0.000000e+00   0.000000e+00     12s

Root relaxation: objective 2.8

In [ ]:
model.printAttr('x')